# < Insert Title here >

# Introduction

# Preliminary Exploratory Data Analysis

# Methods

Each winner and loser of a match will have their stats mutated into three percentage predictors which will be used to Classify whether or not that player was the Winner or Loser of the match. The first predictor calculates successful saved breakpoints while the second and third predictors calculate serves and whether or not that Player won a point on their second serve or received a double fault.

We can apply these values by using scatterplot graphs to compare the Winner and Loser stats for each match and visualize a trend. For example, we can compare the amount of points won on the second serve versus the number of double faults and have each point colored on whether or not that player won or lost the match.

# Expected outcomes and significance